# Cases in Malaysia
> Updates on the respiratory illness that has infected more than one million people and killed tens of thousands.

- toc:false
- branch: master
- badges: false
- hide_colab_badge: true
- comments: false